In [ ]:
from pathlib import Path
import json
import pandas as pd

In [ ]:
list_of_files=sorted(Path('../../twister-out').glob('**/compile_commands.json'))


In [ ]:
print(f"Found {len(list_of_files)} different applications")

In [ ]:
import re
import subprocess
# Regular expression to match the directory name after "twister-out", considering potential trailing slash
pattern = r"twister-out/(?P<platform>[^/]*)/(?P<test_dir>.*)/compile_commands.json"  # Added optional trailing slash to handle both cases

header_pattern = r""
c_files = []

WORKSPACE_DIR=Path('../../').resolve()
print(WORKSPACE_DIR)
counter=1

for f in list_of_files:
    
    file_path = str(f.resolve())

    # Use re.search to find the match
    match = re.search(pattern, file_path)

    # Extract the target directory name if there is a match
    if match:
     platform_name = match.group("platform")
     test_suite_dir = match.group("test_dir")
    else:
     continue

    print(f"Looking at {platform_name} - {test_suite_dir} ({counter}/{len(list_of_files})")
    counter = counter + 1
    
    f = f.resolve()

    with open(f,"r") as text:
        data = json.load(text)

        # each data is a single invocation of the C compiler, the 'file' attribute holds the name of the c-file that is compiled
        h_files = []
        for d in data:
            if(d['file'].endswith('empty_file.c')):
                continue
            
            p=Path(d['file'])

            c_files.append([platform_name, 
                            test_suite_dir, 
                            str(p.relative_to(Path('../..' ).resolve())) ])
            command = f"ninja -C {f.parent} -t deps {d['output']}".split()
            
            result = subprocess.run(command, capture_output=True, text=True)
            
            for line in result.stdout.splitlines():
                if line.endswith(".h"):
                    if line.strip() not in h_files:
                        h_files.append(line.strip())                        
                        h_path = Path(line.strip()).absolute()
                        
                        c_files.append([platform_name, 
                                        test_suite_dir, 
                                        str(h_path.relative_to(WORKSPACE_DIR) ) ])


df = pd.DataFrame(c_files, columns = ['platform', 'test_name', 'file'])
df.to_csv('cortex-m_kernel_tests_and_samples.csv')

In [ ]:
files=df['file']

In [ ]:
files=files.drop_duplicates()
len(files)

In [ ]:
files.to_csv('cortex-m_kernel_tests_and_samples_unique_files.csv')

In [ ]:
platform='mps2_an385'
test_name='samples/basic/minimal/sample.minimal.no-mt-no-timers.arm'
filter = (df['platform'].str.startswith(platform)) & (df['test_name'].str.startswith(test_name))

unique_files = df[filter].drop_duplicates(subset='file')

unique_files.file[unique_files.file.str.startswith('zephyr/')].sort_values()


In [ ]:
#unique_files[unique_files['file'].str.startswith('zephyr/')].sort_values(by='file').to_csv('qemu_cortex_m3_hello_world_samples_intree_files.csv',columns=['file'], index=False)
unique_files.sort_values(by='file').to_csv('qemu_cortex_m3_hello_world_samples_intree_files.csv',columns=['file'], index=False)

In [ ]:
df.test_name.unique()